In [1]:
import pandas as pd
import sys
sys.path.append('./utils')
from utils import *
from rank import *
from process import * 
from tqdm.notebook import tqdm
import seaborn as sns

# データ保存先
data_folda = '../data/'

* masterからデータを追加

In [2]:

### master data から加えるデータを定義
add_list=['name',"prefecture_name",'head_branch','opened_on', 'created_at','genre_first_name', 
          'genre_second_name','genre_third_name','northlatitude', 'eastlongitude',
          'net_reservation_flg','price_range_lunch_owner','price_range_dinner_owner', 
          'price_range_lunch_user','price_range_dinner_user',
          # 20240312追加
          'all_photo_count','photo_food_count','photo_drink_count',
          'photo_interior_count','photo_exterior_count','official_info_flg','max_visit_month']

# make_timeseries_data.ipynb で作成したデータを読み込む
pseudo_time_series=pd.read_pickle(data_folda+'skeleton_train.pkl')

# 直近三カ月の閉店率を計算して保存
# pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-08-31']
# close_ratios=calculate_close_ratio(pseudo_time_series, data_folda)
# close_ratios=pd.read_pickle(data_folda+'close_ratios.pkl')

# 三カ月の閉店率を読み込み
close_ratios=pd.read_pickle(data_folda+'close_ratios.pkl')

pseudo_time_series_test3=pd.read_pickle(data_folda+'skeleton_test_3month_11.pkl')
pseudo_time_series_test3 = add_master_data(pseudo_time_series_test3, data_folda, add_list, "restaurant_data.csv")
pseudo_time_series_test3=pd.merge(pseudo_time_series_test3,close_ratios,on=["base_date","genre_first_name"],how="left")
pseudo_time_series_test3.to_pickle(data_folda+'skeleton_test_3month_master.pkl')


In [ ]:
pseudo_time_series.columns.values

array(['restaurant_id', 'original_id', 'closed_date', 'base_date',
       'target3', 'target6', 'eval_period', 'id', 'name',
       'prefecture_name', 'head_branch', 'opened_on', 'created_at',
       'genre_first_name', 'genre_second_name', 'genre_third_name',
       'northlatitude', 'eastlongitude', 'net_reservation_flg',
       'price_range_lunch_owner', 'price_range_dinner_owner',
       'price_range_lunch_user', 'price_range_dinner_user',
       'all_photo_count', 'photo_food_count', 'photo_drink_count',
       'photo_interior_count', 'photo_exterior_count',
       'official_info_flg', 'real_opened_date', 'openning_days',
       'days_from_lastvisit', 'close_ratio_genre'], dtype=object)

* V_GOOGLE からデータを追加

In [5]:

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month_11.pkl')
add_list=["restaurant_id","overview.aggregateRating","overview.reviewCount","general_rank"]
pseudo_time_series_test = add_v_google(pseudo_time_series_test, data_folda, add_list)
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_3month_google.pkl')

del  pseudo_time_series_test

/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

* V_RETTY からデータを追加

In [6]:
#追加する変数をリストで定義
add_list=["restaurant_id",'infos.isOfficial', 
       'infos.familiar.users.count', 'infos.familiar.stars',
       'infos.onlineReservation_RETTY', 'infos.access.transferTime1','infos.access.transferTime2','infos.access.transferTime3',
       'infos.numberOfSeats.value', 'infos.counterSeats.value',
       'infos.privateRoom.value', 'infos.updateInfo.firstReviewDate',
       'infos.wantToGo', 'infos.updateInfo.lastReviewDate',
       'infos.went', 'infos.rateByWent', 'infos.excellent', 'infos.coupon',
       'infos.photoCount.Photograph', 'infos.photoCount.Cooking',
       'infos.photoCount.Interior', 'infos.photoCount.Exterior',
       'infos.photoCount.Menu', 'infos.reviewCount.Review',
       'infos.reviewCount.Lunch', 'infos.reviewCount.Dinner',
       'menus.course_list_count',
       'menus.dishes_list_count', 'menus.lunch_list_count',
       'menus.drink_list_count', 'menus.takeout_list_count',
       'infos.familiar.users.genge','infos.reserveEntireStore',"general_rank_RETTY"]
 

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month_11.pkl')
pseudo_time_series_test = add_v_retty(pseudo_time_series_test, data_folda, add_list, master_name="V_RETTY",read_from_snowflake=False)
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_3month_retty.pkl')

del  pseudo_time_series_test

/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

* HOTPEPPER からデータを追加

In [7]:
add_list=['restaurant_id','infos.aggregateRating','infos.ratingReview',
          'infos.satisfaction.percentage1','infos.satisfaction.percentage2','infos.satisfaction.percentage3',
          'infos.couponCount','infos.onlineReservation_HOTPEPPER','infos.ownerRegistration','infos.ticket',
          'photos.allCount','photos.appearanceCount','photos.drinkCount','photos.foodCount','photos.otherCount',
          'photos.postCount','menus.courseListCount']


pseudo_time_series_test3 = pd.read_pickle(data_folda+'skeleton_test_3month_11.pkl')
pseudo_time_series_test3= reduce_mem_usage(pseudo_time_series_test3)
pseudo_time_series_test3 = add_v_hotpepper(pseudo_time_series_test3, data_folda, add_list, master_name="V_HOTPEPPER",read_from_snowflake=False) 
pseudo_time_series_test3.to_pickle(data_folda+'skeleton_test_3month_hotpepper.pkl')

del pseudo_time_series_test3

/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

* V_TRIPADVISOR から特徴量を追加

In [9]:
def add_v_tripadvisor(pseudo_time_series, data_folda, add_list, master_name="V_TRIPADVISOR",read_from_snowflake=False):
    pd_sf = read_table(data_folda,master_name,columns="all",read_snowflake=read_from_snowflake,col="*")

    # general rank を与える
    ranker=rank.Ranker()
    pd_sf=ranker.add_general_rank(pd_sf)
    pd_sf=select_general_rank(pd_sf)
    pd_sf.rename(columns={"general_rank":"general_rank_TRIPADVISOR"},inplace=True)

    pd_sf.rename(columns={"infos.aggregateRating":"infos.aggregateRating_TRIPADVISOR"},inplace=True)
    #add_list=["restaurant_id","infos.aggregateRating_TRIPADVISOR","infos.ratingDetails.Meal","infos.ratingDetails.Service","infos.ratingDetails.Price","infos.ratingDetails.Ambience","infos.reviewCount","infos.qa_count"]


    pd_sf_drop_dum=pd_sf[add_list]

    # oficial_info_flg を categorical に変換
    #pd_sf_drop_dum["infos.official_info_flg"]=pd_sf_drop_dum["infos.official_info_flg"].astype("category")
    pseudo_time_series=pseudo_time_series[["base_date","restaurant_id"]]
    temp=pd.merge(pseudo_time_series,pd_sf_drop_dum,on="restaurant_id",how="left")
    
    return temp

In [10]:
add_list=["restaurant_id","general_rank_TRIPADVISOR",'infos.aggregateRating_TRIPADVISOR',"infos.reviewCount","infos.qa_count"]

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month_11.pkl')
pseudo_time_series_test = reduce_mem_usage(pseudo_time_series_test)
pseudo_time_series_test = add_v_tripadvisor(pseudo_time_series_test, data_folda, add_list, master_name="V_TRIPADVISOR",read_from_snowflake=False) 
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_3month_tripadvisor.pkl')

del pseudo_time_series_test

/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

* V_HITOSARA から特徴量を追加

In [11]:
# hitosara premium を追加
add_list=["restaurant_id","infos.premium","general_rank_HITOSARA","infos.onlineReservation_HITOSARA"]

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month_11.pkl')
pseudo_time_series_test= reduce_mem_usage(pseudo_time_series_test)
pseudo_time_series_test = add_v_hitosara(pseudo_time_series_test, data_folda, add_list, master_name="V_HITOSARA",read_from_snowflake=False) 
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_3month_hitosara.pkl')

del pseudo_time_series_test

/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

* v_review_google を追加

In [12]:
# レビューの数
pd_sf=pd.read_csv(data_folda+'feature_aggregation.csv')
# クロール時のこれまでの全レビューのスコアの平均
pd_sf2=pd.read_csv(data_folda+'restaurant_id_sentiment_score.csv')

pd_sf=pd_sf[pd_sf['BASE_DATE']>='2022-12-31']
pd_sf.rename(columns={"BASE_DATE":"base_date","RESTAURANT_ID":"restaurant_id"},inplace=True)
pd_sf["base_date"]=pd.to_datetime(pd_sf["base_date"]).dt.tz_localize(None)

pseudo_time_series = pd.read_pickle(data_folda+'skeleton_test_3month_11.pkl')
pseudo_time_series=pseudo_time_series[["base_date","restaurant_id"]]
temp=pd.merge(pseudo_time_series,pd_sf,on=["base_date","restaurant_id"],how="left")
temp=pd.merge(temp,pd_sf2,on=["restaurant_id"],how="left")
temp.to_pickle(data_folda+'skeleton_test_3month_feature_aggregation.pkl')


del temp, pseudo_time_series

* 近くの店の閉店率

In [13]:
add_list=['restaurant_id', 'base_date', 'close_ratio_near_restaurant_three_months']
# folda + close_ratio_near_restaurant202*.pkl を全部読み込んで結合
close_ratios=get_close_ratio(data_folda)

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month_11.pkl')
pseudo_time_series_test = pd.merge(pseudo_time_series_test,close_ratios,on=["restaurant_id","base_date"],how="left")
pseudo_time_series_test[add_list].to_pickle(data_folda+'skeleton_test_3month_near_close_ratio.pkl')

del pseudo_time_series_test

../data/close_ratio_near_restaurant20230331.pkl
(875314, 3)
../data/close_ratio_near_restaurant20230630.pkl
(1750628, 3)
../data/close_ratio_near_restaurant20230930.pkl
(2625942, 3)
../data/close_ratio_near_restaurant20230430.pkl
(3501256, 3)
../data/close_ratio_near_restaurant20230731.pkl
(4376570, 3)
../data/close_ratio_near_restaurant20230531.pkl
(5251884, 3)
../data/close_ratio_near_restaurant20230831.pkl
(6127198, 3)
../data/close_ratio_near_restaurant20231031.pkl
(7002512, 3)
../data/close_ratio_near_restaurant20231130.pkl
(7877826, 3)


* 近くの駅の数

In [14]:
staton=pd.read_csv(data_folda+"/station_address/station20240318free.csv")
station_cd_count=staton.groupby('station_g_cd').size().reset_index(name='count').sort_values('count', ascending=False)
# 同じ station_cd の数をカウントして、付与する
station=pd.merge(staton,station_cd_count,on='station_g_cd',how='left')

# near_station の駅id(station_cd) と距離（dist_from_station）に オリジナルファイル（station） から 乗り換え可能駅数（count） を付与
near_station = pd.read_pickle(data_folda+'train_with_station_dist.pkl')
near_station = near_station[["restaurant_id","station_cd","dist_from_station"]]
near_station=pd.merge(near_station,station[["station_cd","count"]], on='station_cd', how='left')[["restaurant_id","dist_from_station","count"]]


# 3ヵ月予測テストデータに最寄り駅情報を付与
pseudo_time_series = pd.read_pickle(data_folda+'skeleton_test_3month_11.pkl')
pseudo_time_series = pseudo_time_series[["base_date","restaurant_id"]]
pseudo_time_series = pd.merge(pseudo_time_series,near_station,on="restaurant_id",how="left")
pseudo_time_series.to_pickle(data_folda+'skeleton_test_3month_near_station.pkl')


In [15]:
concat_list=["skeleton_test_3month_master.pkl","skeleton_test_3month_hotpepper.pkl","skeleton_test_3month_google.pkl","skeleton_test_3month_retty.pkl","skeleton_test_3month_tripadvisor.pkl"
             ,"skeleton_test_3month_hitosara.pkl","skeleton_test_3month_feature_aggregation.pkl","skeleton_test_3month_near_station.pkl","skeleton_test_3month_near_close_ratio.pkl"]
#pseudo_time_series=pseudo_time_series.drop(["base_date","restaurant_id"],axis=1)
df=pd.DataFrame()
for i, fn in enumerate(concat_list):
    print(fn)
        
    df_=pd.read_pickle(data_folda+fn)
    if(i==0):
        pass
    else:
        df_=df_.drop(["base_date","restaurant_id"],axis=1)
    
    df=pd.concat([df,df_],axis=1)
df.to_pickle(data_folda+'test3_with_all_11.pkl')

skeleton_test_3month_master.pkl
skeleton_test_3month_hotpepper.pkl
skeleton_test_3month_google.pkl
skeleton_test_3month_retty.pkl
skeleton_test_3month_tripadvisor.pkl
skeleton_test_3month_hitosara.pkl
skeleton_test_3month_feature_aggregation.pkl
skeleton_test_3month_near_station.pkl
skeleton_test_3month_near_close_ratio.pkl
